## Assumptions: W = 0, V != 0, N is even

In [7]:
from scipy.optimize import root

#### Experimenting with Scipy Root

In [8]:
import numpy as np
def fun(x):
    return [x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0, 
            0.5 * (x[1] - x[0])**3 + x[1]]

def jac(x):
    return np.array([[1 + 1.5 * (x[0] - x[1])**2,
                      -1.5 * (x[0] - x[1])**2],
                     [-1.5 * (x[1] - x[0])**2,
                      1 + 1.5 * (x[1] - x[0])**2]])

In [9]:
fun([0,0])

[-1.0, 0.0]

In [10]:
jac([0,0])

array([[ 1., -0.],
       [-0.,  1.]])

In [11]:
root(fun, [0, 0], jac=jac)

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.110e-16  0.000e+00]
       x: [ 8.412e-01  1.588e-01]
  method: hybr
    nfev: 12
    njev: 1
    fjac: [[ 8.991e-01 -4.377e-01]
           [ 4.377e-01  8.991e-01]]
       r: [ 2.167e+00 -1.037e+00  1.106e+00]
     qtf: [-1.196e-11  4.128e-12]

In [12]:
root(fun, [0, 0])

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.110e-16  0.000e+00]
       x: [ 8.412e-01  1.588e-01]
  method: hybr
    nfev: 14
    fjac: [[-8.991e-01  4.377e-01]
           [-4.377e-01 -8.991e-01]]
       r: [-2.167e+00  1.037e+00 -1.106e+00]
     qtf: [ 1.196e-11 -4.128e-12]

## Solving for Bethe Roots

In [13]:
LARGE_INT = 9999
POKE = 1e-4
def bethe_func_N_even_mini(E, l, M, V, v):
    assert len(E) == M, f'Incorrect input vector length (expected: {M}, actual: {len(E)})'
    N = 2*M + 2*v
    
    left = 1
    left_denom = (N * (E[l]**2 - 1))
    if left_denom == 0:
        return LARGE_INT
    left += 2*(1 + 2*v)*V*E[l] / left_denom
    right = 0
    for n in range(0, M):
        if n == l:
            continue
        right_denom = (E[l] - E[n])
        if right_denom == 0:
            right_denom += POKE
            
        right += (1 + E[l]*E[n])/right_denom

    return left + (2*V*right / N)

def bethe_func_N_odd_mini(E, l, M, V, v_diff):
    assert len(E) == M, f'Incorrect input vector length (expected: {M}, actual: {len(E)})'
    N = 2*M + 1 # v_a + v_b always = 1 in odd case
    
    left = 1
    left_denom = (N * (E[l]**2 - 1))
    if left_denom == 0:
        return LARGE_INT
    left += (2*(1 + 2*v)*V*E[l] + V*v_diff*(1 + E[l]**2))/ left_denom
    right = 0
    for n in range(0, M):
        if n == l:
            continue
        right_denom = (E[l] - E[n])
        if right_denom == 0:
            right_denom += POKE
            
        right += (1 + E[l]*E[n])/right_denom

    return left + (2*V*right / N)

In [14]:
from math import sqrt
class Bethe_Analytic:
    def m_1_solve(V, v):
        M = 1
        N = 2*M + 2*v
        num = lambda s: V*(1+2*v) + s*sqrt(V*V*(1+2*v)**2 + N**2)
        solns = [num(s)/(-1*N) for s in [1,-1]]
        return solns

    def m_2_solve(V,v):
        num = lambda s: V*(1+2*v) + s*sqrt(V*V*(1+2*v)**2 + 4*(1+v)**2)
        solns = [num(s)/(-2*(1+v)) for s in [1,-1]]
        return solns


def bethe_func_N_even(E, M, consts):
    assert len(E) == M, f'Incorrect input vector length (expected: {M}, actual: {len(E)})'
    V, v = consts
    
    full_out = [bethe_func_N_even_mini(E, l, M, V, v) for l in range(0, M)]
    return np.array(full_out)

def bethe_func_N_odd(E, M, consts):
    assert len(E) == M, f'Incorrect input vector length (expected: {M}, actual: {len(E)})'
    V, v_diff = consts
    
    full_out = [bethe_func_N_even_mini(E, l, M, V, v_diff) for l in range(0, M)]
    return np.array(full_out)


### M=1 Testing

In [15]:
M = 1
bethe_func_N_even([0]*M, M, consts=(1,1))

array([1.])

In [16]:
bethe_func_N_even([0]*M, M, consts=(1,0))

array([1.])

In [17]:
init = [0]*M
V = 1
root(bethe_func_N_even, init, args=(M, (V,0)), method='df-sane')

 message: successful convergence
 success: True
     fun: [-5.376e-10]
       x: [ 6.180e-01]
     nit: 14
    nfev: 29
  method: df-sane

In [18]:
root(bethe_func_N_even, init, args=(M, (V,1)), method='df-sane')

 message: successful convergence
 success: True
     fun: [ 5.574e-13]
       x: [ 5.000e-01]
     nit: 9
    nfev: 12
  method: df-sane

In [19]:
[Bethe_Analytic.m_1_solve(V,v) for v in range(0,2)]

[[-1.618033988749895, 0.6180339887498949], [-2.0, 0.5]]

In [20]:
[Bethe_Analytic.m_2_solve(V,v) for v in range(0,2)]

[[-1.618033988749895, 0.6180339887498949], [-2.0, 0.5]]

### Testing Larger Ms

In [21]:
max_M = 6
better_init = lambda M: [i/M for i in range(M)]

# Manually check by printing results and noticing proximity to 0
def check_roots(roots, M, outcomes):
    for v in range(0,2):
        print(outcomes[v], bethe_func_N_even(E=roots[v], M=M, consts=(V,v))) 

# Check all Ms outputs
def verify_test(max_M, all_roots, all_outcomes):
    for M in range(1, max_M+1):
        check_roots(all_roots[M-1], M, all_outcomes[M-1])
        print()
        
def test_spectral_solver(max_M, method='hybr', init_strat=None, N_even=True):
    all_roots = []
    all_outcomes = []
    
    for M in range(1, max_M+1):
        init = [.5]*M
        if init_strat:
            init = init_strat(M)
        roots = []
        if N_even:
            for v in range(0,2):
                roots.append(root(bethe_func_N_even, init, args=(M, (V,v)), method=method))
        else:
            for v_diff in [-1,1]:
                roots.append(root(bethe_func_N_even, init, args=(M, (V,v_diff)), method=method))
            
        all_outcomes.append([r.success for r in roots])
        all_roots.append([r.x for r in roots])
        print(f'M={M} Roots:\t{roots}')
        print()

    return all_roots, all_outcomes

hybr_roots, hybr_outcomes = test_spectral_solver(max_M, init_strat=better_init)

M=1 Roots:	[ message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 1.000e+00]
       x: [ 1.374e-10]
  method: hybr
    nfev: 15
    fjac: [[-1.000e+00]]
       r: [ 1.000e+00]
     qtf: [-1.000e+00],  message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00]
       x: [ 5.000e-01]
  method: hybr
    nfev: 13
    fjac: [[-1.000e+00]]
       r: [ 3.333e+00]
     qtf: [-2.863e-13]]

M=2 Roots:	[ message: The solution converged.
 success: True
  status: 1
     fun: [ 2.613e-11 -8.455e-11]
       x: [ 2.107e-01  8.764e-01]
  method: hybr
    nfev: 30
    fjac: [[-7.745e-01  6.326e-01]
           [-6.326e-01 -7.745e-01]]
       r: [ 3.368e+00 -1.184e+01  1.276e+01]
     qtf: [-3.172e-08  2.106e-08],  message: The solution converged.
 success: True
  status: 1
     fun: [-5.810e-13 -9.125e-13]
       x: [ 2.337e-01  7.546e-01]
  method: hybr
    nfev: 22
  

In [22]:
verify_test(max_M, hybr_roots, hybr_outcomes)

False [1.]
True [0.]

True [ 2.61319855e-11 -8.45472581e-11]
True [-5.80979709e-13 -9.12492304e-13]

True [-7.54927898e-10  4.91616636e-10  1.89760940e-09]
True [ 6.26609875e-13 -1.62192482e-12  4.36095604e-13]

False [0.24533802 0.15055857 0.02931132 1.37727494]
True [ 1.41191459e-09 -1.23098975e-09 -6.64708177e-10  2.95137248e-11]

True [-3.88911570e-10 -6.82436663e-10 -1.20706346e-08  3.27973053e-08
  2.12487059e-08]
True [ 2.48332910e-10 -6.32366937e-10  3.25821259e-10 -2.94934119e-10
 -1.62473190e-09]

True [ 1.93139060e-10  4.06469192e-10  4.44583703e-09 -1.95968081e-08
  1.72400587e-08  4.06709866e-09]
True [ 1.30162547e-12 -3.21975779e-12  2.17137419e-12 -1.64901426e-12
  1.02701875e-12 -8.21875901e-12]



In [23]:
df_roots, df_outcomes = test_spectral_solver(max_M, method='df-sane', init_strat=better_init)

M=1 Roots:	[ message: successful convergence
 success: True
     fun: [-5.376e-10]
       x: [ 6.180e-01]
     nit: 14
    nfev: 29
  method: df-sane,  message: successful convergence
 success: True
     fun: [ 5.574e-13]
       x: [ 5.000e-01]
     nit: 9
    nfev: 12
  method: df-sane]

M=2 Roots:	[ message: successful convergence
 success: True
     fun: [-1.553e-08  2.139e-09]
       x: [ 7.808e-01 -1.281e+00]
     nit: 15
    nfev: 17
  method: df-sane,  message: successful convergence
 success: True
     fun: [-2.217e-09 -6.033e-10]
       x: [ 2.337e-01  7.546e-01]
     nit: 19
    nfev: 28
  method: df-sane]

M=3 Roots:	[ message: too many function evaluations required
 success: False
     fun: [ 3.243e-01 -9.856e-02  9.350e-01]
       x: [-1.174e+00  8.162e-01  3.075e+01]
     nit: 254
    nfev: 1000
  method: df-sane,  message: successful convergence
 success: True
     fun: [ 2.328e-09  8.573e-10  3.598e-10]
       x: [ 8.019e-02  5.176e-01  8.564e-01]
     nit: 28
    nfev:

In [24]:
verify_test(max_M, df_roots, df_outcomes)

True [-5.37584643e-10]
True [5.57442981e-13]

True [-1.55260065e-08  2.13882416e-09]
True [-2.21679719e-09 -6.03311401e-10]

False [ 0.32433423 -0.09855952  0.93499893]
True [2.32757880e-09 8.57328541e-10 3.59798413e-10]

False [ 3.80047330e-02 -9.33923190e-05  5.14410427e-01 -2.29206027e-03]
True [-5.01250885e-09  1.58381923e-08 -2.56594063e-09 -4.64122962e-10]

False [ 9.00959230e-01  2.28424547e-04  4.54098446e-04  3.27990087e-03
 -3.76759235e-02]
True [-2.07415907e-09 -1.46556003e-08  1.50153513e-08 -3.88322008e-09
  7.72099967e-10]

False [ 9.40332472e-04  5.69200030e-03  1.06540941e+00 -7.09693322e-02
  3.17824162e-04  1.69879129e-03]
True [-1.09051657e-10  2.93195046e-09  3.01833153e-09 -1.02970765e-08
  1.55946100e-09  1.03311137e-09]



#### Odd Ns

In [25]:
hybr_roots, hybr_outcomes = test_spectral_solver(max_M, init_strat=better_init, N_even=False)

M=1 Roots:	[ message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 9.999e+03]
       x: [ 0.000e+00]
  method: hybr
    nfev: 15
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 9.999e+03],  message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00]
       x: [ 5.000e-01]
  method: hybr
    nfev: 13
    fjac: [[-1.000e+00]]
       r: [ 3.333e+00]
     qtf: [-2.863e-13]]

M=2 Roots:	[ message: The solution converged.
 success: True
  status: 1
     fun: [-2.700e-11  7.457e-11]
       x: [-6.180e-01  1.618e+00]
  method: hybr
    nfev: 27
    fjac: [[-8.458e-01 -5.334e-01]
           [ 5.334e-01 -8.458e-01]]
       r: [-3.038e+00 -8.713e-01 -1.006e+00]
     qtf: [-8.704e-10 -3.921e-09],  message: The solution converged.
 success: True
  status: 1
     fun: [-5.810e-13 -9.125e-13]
       x: [ 2.337e-01  7.546e-01]
  method: hybr
    nfev: 22
  

In [26]:
verify_test(max_M, hybr_roots, hybr_outcomes)

False [1.]
True [0.]

True [1.5 1.5]
True [-5.80979709e-13 -9.12492304e-13]

True [ 1.2967738  -0.30053675  4.00376294]
True [ 6.26609875e-13 -1.62192482e-12  4.36095604e-13]

False [ 0.62451808 -0.21310704  3.6493326   1.39718961]
True [ 1.41191459e-09 -1.23098975e-09 -6.64708177e-10  2.95137248e-11]

True [ 0.35836346  0.12028445 -0.15691019 -1.66037677 10.33863905]
True [ 2.48332910e-10 -6.32366937e-10  3.25821259e-10 -2.94934119e-10
 -1.62473190e-09]

False [0.31416121 0.35315494 1.18995685 2.53294861 1.20680217 1.24533449]
True [ 1.30162547e-12 -3.21975779e-12  2.17137419e-12 -1.64901426e-12
  1.02701875e-12 -8.21875901e-12]

